# Installing the Dependencies


In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118


In [2]:
!pip install transformers requests beautifulsoup4 pandas numpy

# Importing Dependencies

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# Instantiate model

In [4]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment') # Load the tokenizer for the bert model.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')# Load the model for sequence classification based on the same BERT model.

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# Encode and Calculate Sentiment

In [6]:
tokens = tokenizer.encode('I hated everything in the menu', return_tensors='pt')

In [7]:
tokens

tensor([[  101,   151, 39487, 10163, 23225, 10104, 10103, 43182,   102]])

In [8]:
#reconverting the tokens
reconv_token = tokenizer.decode(tokens[0])

In [9]:
reconv_token

'[CLS] i hated everything in the menu [SEP]'

In [10]:
result = model(tokens)

In [12]:
result #retorna um array de 5 valores contendo a probabilidade de cada nota

SequenceClassifierOutput(loss=None, logits=tensor([[ 1.9038,  1.3139,  0.1833, -1.3484, -1.7061]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [16]:
highest = torch.argmax(result.logits) #Retrieves the highest probability.
highest

tensor(0)

In [17]:
int(highest)+1

1

# Collecting Yelp Reviews

In [18]:
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')

In [19]:
soup = BeautifulSoup(r.text, 'html.parser')

In [20]:
regex = re.compile('.*comment.*')

In [21]:
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [22]:
reviews

["Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 "Six of us met here for breakfast before our walk to Manly. We were enjoying visiting with each other so much that I apologize for not taking any photos. We all enjoyed our food, as well as our coffee and tea drinks.We were greeted immediately by a friendly server asking if we would like to sit inside or out. We said we would like inside, but weren't exactly sure how many were joining us yet- at least 4. 

# Load Reviews into DataFrame and Score

In [23]:
import numpy as np
import pandas as pd

In [24]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [27]:
df.head()

,review
0,Very cute coffee shop and restaurant. They hav...
1,Six of us met here for breakfast before our wa...
2,"Great service, lovely location, and really ama..."
3,Great place with delicious food and friendly s...
4,Some of the best Milkshakes me and my daughter...


In [28]:
df['review'].iloc[0]

"Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!"

In [29]:
def sentiment_analysis(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1



In [31]:
#testing the function

sentiment_analysis(df['review'].iloc[1])

4

In [33]:
# aplying the function to all data
df['sentiment'] = df['review'].apply(lambda x: sentiment_analysis(x[:512]))

In [34]:
df.head()

,review,sentiment
0,Very cute coffee shop and restaurant. They hav...,4
1,Six of us met here for breakfast before our wa...,4
2,"Great service, lovely location, and really ama...",5
3,Great place with delicious food and friendly s...,5
4,Some of the best Milkshakes me and my daughter...,5


In [35]:
df['review'].iloc[3]

'Great place with delicious food and friendly staff. It is small but has outdoor seating and a relaxed ambiance. Perfect place to enjoy a cup of coffee. I am visiting Sydney for the first time but this place seems like is a local favorite.'